In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [52]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("../general/ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [98]:
demo = pd.read_csv("PTDEMOG.csv").rename(columns={"PHASE":"Phase"})

In [54]:
neuro = pd.read_csv("NEUROEXM.csv").rename(columns={"PHASE":"Phase"})

In [55]:
neuro.columns

Index(['Phase', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [56]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [57]:
clinical.head()

,RID,SUBJECT_KEY,Phase,VISCODE,VISCODE2,EXAMDATE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,AGE,DX,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP,update_stamp
0,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,1,16.0,2,5,80.4682,1.0,0.090,0.002,0.460,0.264,2021-04-14 13:05:55.0
1,2,ADNI_011_S_0002,ADNI2,v21,m96,2013-09-09,1,16.0,2,5,82.4422,1.0,0.288,-0.495,0.460,NaN,2021-04-14 13:05:56.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,1,16.0,2,5,77.4073,1.0,0.344,0.419,0.535,NaN,2021-04-14 13:05:56.0
3,2,ADNI_011_S_0002,ADNI2,v41,m120,2015-09-22,1,16.0,2,5,84.4764,1.0,0.382,-0.549,0.595,-0.333,2021-04-14 13:05:56.0
4,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,1,16.0,2,5,79.4771,1.0,0.101,0.066,0.620,0.264,2021-04-14 13:05:56.0


In [58]:
diag["Subject"].value_counts()

,Subject,GroupN,Phase
0,002_S_0295,0.0,ADNI1
1,002_S_0295,0.0,ADNI2
2,002_S_0413,0.0,ADNI1
3,002_S_0413,0.0,ADNI2
4,002_S_0413,0.0,ADNI3
...,...,...,...
3305,941_S_6580,0.0,ADNI3
3306,941_S_6581,0.0,ADNI3
3307,941_S_6607,0.0,ADNI3
3308,941_S_6803,1.0,ADNI3


In [96]:
comb = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})
comb["PTID"]= comb["SUBJECT_KEY"].str.replace("ADNI_", "").str.replace("s", "S")
comb = comb[["RID", "PTID" , "Phase"]]

In [60]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [61]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [62]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [63]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [64]:
m["PTID"].value_counts()

PTID
131_S_0384    15
141_S_0697    15
116_S_0752    14
067_S_4072    14
011_S_0023    14
              ..
141_S_1255     1
109_S_4260     1
128_S_4603     1
022_S_1097     1
109_S_2237     1
Name: count, Length: 671, dtype: int64

In [65]:
t = m
pd.set_option('display.max_columns', None)
print(t)

       RID        PTID  Phase VISCODE VISCODE2     VISDATE  PTSOURCE  \
0       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
1       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
2       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
3       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
4       21  011_S_0021  ADNI1      sc       sc  2005-09-28         1   
...    ...         ...    ...     ...      ...         ...       ...   
3370  4696  018_S_4696  ADNI2     v01       sc  2012-04-19         1   
3371  4696  018_S_4696  ADNI2     v01       sc  2012-04-19         1   
3372  4707  005_S_4707  ADNI2     v01       sc  2012-04-26         1   
3373  4707  005_S_4707  ADNI2     v01       sc  2012-04-26         1   
3374  4707  005_S_4707  ADNI2     v01       sc  2012-04-26         1   

      PTGENDER    PTDOB  PTDOBYY  PTHAND  PTMARRY  PTEDUCAT  PTWORKHS  PTWORK  \
0          2.0  02/1933   1933.0     1.0      3.0     

In [66]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp",  "PTSOURCE", "PTDOB","DX"], axis=1) 

In [67]:
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR',
       'PTHOME', 'PTTLANG', 'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX',
       'PTETHCAT', 'PTRACCAT', 'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK',
       'PTNLANG', 'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM',
       'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1',
       'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2',
       'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3',
       'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4',
       'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5',
       'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6',
       'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN',
       'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', '

In [68]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

/tmp/ipykernel_6292/861819902.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  t = t.fillna(-4)


In [69]:
len(t.columns)
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'SUBJECT_KEY', 'EXAMDATE', 'AGE', 'PHC_MEM', 'PHC_EXF',
       'PHC_LAN', 'PHC_VSP', 'GroupN'],
      dtype='object')

In [70]:
# t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper","bookkeeper").str.replace("cpa", "accounting")

In [71]:
# t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

In [72]:
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

In [73]:
# cond = t['PTWORK'].value_counts()
# threshold = 10
# t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [74]:
# categorical = ['PTGENDER', 'PTWORK',
#  'PTHOME',
#  'PTMARRY',
#  'PTEDUCAT',
#  'PTPLANG',
#  'NXVISUAL',
#  'PTNOTRT',
#  'NXTREMOR',
#  'NXAUDITO',
#  'PTHAND']

# REMOVE PTWORK FOR NOW
categorical = ['PTGENDER',
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [75]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [76]:
# text = ["PTWORK", "CMMED"]

# REMOVE PTWORK FOR NOW
text = ["CMMED"]

In [77]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

,NXSENSOR,GroupN,NXNERVE,PTTLANG,PTRACCAT,PHC_LAN,NXMOTOR,NXABNORM,PHC_EXF,PHC_MEM,NXTENDON,NXPLANTA,PTADDX,PTRTYR,PTCOGBEG,NXFINGER,EXAMDATE,PTDOBYY,NXHEEL,PTETHCAT,VISDATE,NXGAIT,PHC_VSP,SUBJECT_KEY,NXOTHER,NXCONSCI,AGE
0,2.0,0.0,1.0,1.0,4,1.813,1.0,1.0,0.401,1.530,1.0,1.0,-4.0,1989.0,-4.0,1.0,2006-11-01,1933.0,1.0,2.0,2005-09-28,1.0,0.264,ADNI_011_S_0021,1.0,1.0,73.7467
1,2.0,0.0,1.0,1.0,4,1.170,1.0,1.0,0.564,1.203,1.0,1.0,-4.0,1989.0,-4.0,1.0,2007-10-31,1933.0,1.0,2.0,2005-09-28,1.0,0.264,ADNI_011_S_0021,1.0,1.0,74.7433
2,2.0,0.0,1.0,1.0,4,1.372,1.0,1.0,0.418,1.254,1.0,1.0,-4.0,1989.0,-4.0,1.0,2006-04-24,1933.0,1.0,2.0,2005-09-28,1.0,-0.333,ADNI_011_S_0021,1.0,1.0,73.2238
3,2.0,0.0,1.0,1.0,4,0.816,1.0,1.0,0.288,1.300,1.0,1.0,-4.0,1989.0,-4.0,1.0,2005-10-24,1933.0,1.0,2.0,2005-09-28,1.0,0.264,ADNI_011_S_0021,1.0,1.0,72.7255
4,2.0,0.0,1.0,1.0,4,1.274,1.0,1.0,0.645,1.620,1.0,1.0,-4.0,1989.0,-4.0,1.0,2008-10-22,1933.0,1.0,2.0,2005-09-28,1.0,-4.000,ADNI_011_S_0021,1.0,1.0,75.7207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,1.0,2.0,1.0,1.0,5,-1.929,1.0,1.0,-2.210,-2.220,1.0,1.0,2006.0,2009.0,2004.0,1.0,2014-04-29,1939.0,1.0,2.0,2012-04-19,1.0,-1.370,ADNI_018_S_4696,1.0,1.0,75.0773
3371,1.0,2.0,1.0,1.0,5,-1.391,1.0,1.0,-1.023,-1.872,1.0,1.0,2006.0,2009.0,2004.0,1.0,2012-05-09,1939.0,1.0,2.0,2012-04-19,1.0,-4.000,ADNI_018_S_4696,1.0,1.0,73.1061
3372,1.0,2.0,1.0,1.0,5,-0.973,1.0,1.0,-1.057,-1.365,2.0,1.0,2011.0,2008.0,2006.0,1.0,2013-06-18,1944.0,1.0,2.0,2012-04-26,1.0,0.264,ADNI_005_S_4707,1.0,1.0,69.1307
3373,1.0,2.0,1.0,1.0,5,-0.730,1.0,1.0,-0.227,-0.744,2.0,1.0,2011.0,2008.0,2006.0,1.0,2012-06-05,1944.0,1.0,2.0,2012-04-26,1.0,0.264,ADNI_005_S_4707,1.0,1.0,68.0958


In [78]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXSENSOR
GroupN
NXNERVE
PTTLANG
PTRACCAT
NXMOTOR
NXABNORM
NXTENDON
NXPLANTA
NXFINGER
NXHEEL
PTETHCAT
NXGAIT
NXOTHER
NXCONSCI


In [79]:
t.columns

Index(['RID', 'PTID', 'Phase', 'VISDATE', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'SUBJECT_KEY', 'EXAMDATE', 'AGE', 'PHC_MEM', 'PHC_EXF',
       'PHC_LAN', 'PHC_VSP', 'GroupN'],
      dtype='object')

In [80]:
# to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY", "PTWRECNT"]

#Remove PTWRECNT for now!! -> not in any .csv file
to_del = ["PTRTYR", "EXAMDATE", "SUBJECT_KEY"]
t = t.drop(to_del, axis=1)

In [81]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

,PTDOBYY,VISDATE,PHC_VSP,PHC_LAN,PHC_EXF,PHC_MEM,AGE,PTADDX,PTCOGBEG
0,1933.0,2005-09-28,0.264,1.813,0.401,1.530,73.7467,-4.0,-4.0
1,1933.0,2005-09-28,0.264,1.170,0.564,1.203,74.7433,-4.0,-4.0
2,1933.0,2005-09-28,-0.333,1.372,0.418,1.254,73.2238,-4.0,-4.0
3,1933.0,2005-09-28,0.264,0.816,0.288,1.300,72.7255,-4.0,-4.0
4,1933.0,2005-09-28,-4.000,1.274,0.645,1.620,75.7207,-4.0,-4.0
...,...,...,...,...,...,...,...,...,...
3370,1939.0,2012-04-19,-1.370,-1.929,-2.210,-2.220,75.0773,2006.0,2004.0
3371,1939.0,2012-04-19,-4.000,-1.391,-1.023,-1.872,73.1061,2006.0,2004.0
3372,1944.0,2012-04-26,0.264,-0.973,-1.057,-1.365,69.1307,2011.0,2006.0
3373,1944.0,2012-04-26,0.264,-0.730,-0.227,-0.744,68.0958,2011.0,2006.0


In [82]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [83]:
#after reviewing the meaning of each column, these are the final ones
# l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
#        'PTMARRY', 'PTEDUCAT', 'PTWORK', 'PTNOTRT', 'PTHOME', 'PTTLANG',
#        'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
#        'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
#        'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
#        'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

# REMOVE PTWORK FOR NOW
l = ['RID', 'PTID', 'GroupN', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [84]:
t[l]

,RID,PTID,GroupN,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,PTHOME,PTTLANG,PTPLANG,PTCOGBEG,PTETHCAT,PTRACCAT,NXVISUAL,NXAUDITO,NXTREMOR,NXCONSCI,NXNERVE,NXMOTOR,NXFINGER,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.530,0.401,1.813,0.264
1,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.203,0.564,1.170,0.264
2,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.254,0.418,1.372,-0.333
3,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.300,0.288,0.816,0.264
4,21,011_S_0021,0.0,ADNI1,2.0,1933.0,1.0,3.0,18.0,1.0,1.0,1.0,1.0,-4.0,2.0,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.620,0.645,1.274,-4.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,4696,018_S_4696,2.0,ADNI2,2.0,1939.0,2.0,1.0,16.0,1.0,1.0,1.0,1.0,2004.0,2.0,5,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-2.220,-2.210,-1.929,-1.370
3371,4696,018_S_4696,2.0,ADNI2,2.0,1939.0,2.0,1.0,16.0,1.0,1.0,1.0,1.0,2004.0,2.0,5,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.872,-1.023,-1.391,-4.000
3372,4707,005_S_4707,2.0,ADNI2,1.0,1944.0,1.0,1.0,14.0,1.0,1.0,1.0,1.0,2006.0,2.0,5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,-1.365,-1.057,-0.973,0.264
3373,4707,005_S_4707,2.0,ADNI2,1.0,1944.0,1.0,1.0,14.0,1.0,1.0,1.0,1.0,2006.0,2.0,5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,-0.744,-0.227,-0.730,0.264


In [85]:
dfs = []

In [86]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [87]:
cat = pd.concat(dfs, axis=1)

In [88]:
t[quant]

,PTDOBYY,VISDATE,PHC_VSP,PHC_LAN,PHC_EXF,PHC_MEM,AGE,PTADDX,PTCOGBEG
0,1933.0,2005-09-28,0.264,1.813,0.401,1.530,73.7467,-4.0,-4.0
1,1933.0,2005-09-28,0.264,1.170,0.564,1.203,74.7433,-4.0,-4.0
2,1933.0,2005-09-28,-0.333,1.372,0.418,1.254,73.2238,-4.0,-4.0
3,1933.0,2005-09-28,0.264,0.816,0.288,1.300,72.7255,-4.0,-4.0
4,1933.0,2005-09-28,-4.000,1.274,0.645,1.620,75.7207,-4.0,-4.0
...,...,...,...,...,...,...,...,...,...
3370,1939.0,2012-04-19,-1.370,-1.929,-2.210,-2.220,75.0773,2006.0,2004.0
3371,1939.0,2012-04-19,-4.000,-1.391,-1.023,-1.872,73.1061,2006.0,2004.0
3372,1944.0,2012-04-26,0.264,-0.973,-1.057,-1.365,69.1307,2011.0,2006.0
3373,1944.0,2012-04-26,0.264,-0.730,-0.227,-0.744,68.0958,2011.0,2006.0


In [89]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,PTHOME_8.0,PTMARRY_-4.0,PTMARRY_1.0,PTMARRY_2.0,PTMARRY_3.0,PTMARRY_4.0,PTMARRY_5.0,PTEDUCAT_-4.0,PTEDUCAT_6.0,PTEDUCAT_7.0,PTEDUCAT_8.0,PTEDUCAT_9.0,PTEDUCAT_10.0,PTEDUCAT_11.0,PTEDUCAT_12.0,PTEDUCAT_13.0,PTEDUCAT_14.0,PTEDUCAT_15.0,PTEDUCAT_16.0,PTEDUCAT_17.0,PTEDUCAT_18.0,PTEDUCAT_19.0,PTEDUCAT_20.0,PTPLANG_-4.0,PTPLANG_1.0,PTPLANG_2.0,PTPLANG_3.0,NXVISUAL_1.0,NXVISUAL_2.0,PTNOTRT_-4.0,PTNOTRT_0.0,PTNOTRT_1.0,PTNOTRT_2.0,NXTREMOR_1.0,NXTREMOR_2.0,NXAUDITO_1.0,NXAUDITO_2.0,PTHAND_-4.0,PTHAND_1.0,PTHAND_2.0,NXSENSOR_1.0,NXSENSOR_2.0,GroupN_0.0,GroupN_1.0,GroupN_2.0,NXNERVE_1.0,NXNERVE_2.0,PTTLANG_-4.0,PTTLANG_1.0,PTTLANG_2.0,PTRACCAT_-4,PTRACCAT_1,PTRACCAT_2,PTRACCAT_3,PTRACCAT_4,PTRACCAT_5,PTRACCAT_6,PTRACCAT_7,NXMOTOR_1.0,NXMOTOR_2.0,NXABNORM_-4.0,NXABNORM_1.0,NXABNORM_2.0,NXTENDON_1.0,NXTENDON_2.0,NXPLANTA_1.0,NXPLANTA_2.0,NXFINGER_1.0,NXFINGER_2.0,NXHEEL_-4.0,NXHEEL_1.0,NXHEEL_2.0,PTETHCAT_-4.0,PTETHCAT_1.0,PTETHCAT_2.0,PTETHCAT_3.0,NXGAIT_-4.0,NXGAIT_1.0,NXGAIT_2.0,NXOTHER_-4.0,NXOTHER_-1.0,NXOTHER_1.0,NXOTHER_2.0,NXCONSCI_1.0,NXCONSCI_2.0
0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False
1,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False
2,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False
3,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False
4,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [90]:
t[["PTID","RID", "Phase", "GroupN"]]

,PTID,RID,Phase,GroupN
0,011_S_0021,21,ADNI1,0.0
1,011_S_0021,21,ADNI1,0.0
2,011_S_0021,21,ADNI1,0.0
3,011_S_0021,21,ADNI1,0.0
4,011_S_0021,21,ADNI1,0.0
...,...,...,...,...
3370,018_S_4696,4696,ADNI2,2.0
3371,018_S_4696,4696,ADNI2,2.0
3372,005_S_4707,4707,ADNI2,2.0
3373,005_S_4707,4707,ADNI2,2.0


In [91]:
c = pd.concat([t[["PTID", "RID", "Phase", "GroupN"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [92]:
c

,PTID,RID,Phase,GroupN,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,PTHOME_8.0,PTMARRY_-4.0,PTMARRY_1.0,PTMARRY_2.0,PTMARRY_3.0,PTMARRY_4.0,PTMARRY_5.0,PTEDUCAT_-4.0,PTEDUCAT_6.0,PTEDUCAT_7.0,PTEDUCAT_8.0,PTEDUCAT_9.0,PTEDUCAT_10.0,PTEDUCAT_11.0,PTEDUCAT_12.0,PTEDUCAT_13.0,PTEDUCAT_14.0,PTEDUCAT_15.0,PTEDUCAT_16.0,PTEDUCAT_17.0,PTEDUCAT_18.0,PTEDUCAT_19.0,PTEDUCAT_20.0,PTPLANG_-4.0,PTPLANG_1.0,PTPLANG_2.0,PTPLANG_3.0,NXVISUAL_1.0,NXVISUAL_2.0,PTNOTRT_-4.0,PTNOTRT_0.0,PTNOTRT_1.0,PTNOTRT_2.0,NXTREMOR_1.0,NXTREMOR_2.0,NXAUDITO_1.0,NXAUDITO_2.0,PTHAND_-4.0,PTHAND_1.0,PTHAND_2.0,NXSENSOR_1.0,NXSENSOR_2.0,GroupN_0.0,GroupN_1.0,GroupN_2.0,NXNERVE_1.0,NXNERVE_2.0,PTTLANG_-4.0,PTTLANG_1.0,PTTLANG_2.0,PTRACCAT_-4,PTRACCAT_1,PTRACCAT_2,PTRACCAT_3,PTRACCAT_4,PTRACCAT_5,PTRACCAT_6,PTRACCAT_7,NXMOTOR_1.0,NXMOTOR_2.0,NXABNORM_-4.0,NXABNORM_1.0,NXABNORM_2.0,NXTENDON_1.0,NXTENDON_2.0,NXPLANTA_1.0,NXPLANTA_2.0,NXFINGER_1.0,NXFINGER_2.0,NXHEEL_-4.0,NXHEEL_1.0,NXHEEL_2.0,PTETHCAT_-4.0,PTETHCAT_1.0,PTETHCAT_2.0,PTETHCAT_3.0,NXGAIT_-4.0,NXGAIT_1.0,NXGAIT_2.0,NXOTHER_-4.0,NXOTHER_-1.0,NXOTHER_1.0,NXOTHER_2.0,NXCONSCI_1.0,NXCONSCI_2.0,PTDOBYY,VISDATE,PHC_VSP,PHC_LAN,PHC_EXF,PHC_MEM,AGE,PTADDX,PTCOGBEG
0,011_S_0021,21,ADNI1,0.0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,1933.0,2005-09-28,0.264,1.813,0.401,1.530,73.7467,-4.0,-4.0
1,011_S_0021,21,ADNI1,0.0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,1933.0,2005-09-28,0.264,1.170,0.564,1.203,74.7433,-4.0,-4.0
2,011_S_0021,21,ADNI1,0.0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,1933.0,2005-09-28,-0.333,1.372,0.418,1.254,73.2238,-4.0,-4.0
3,011_S_0021,21,ADNI1,0.0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,True,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False,True,False,True,False,False,True,False,False,False,True,False,False,True,False,False,False,True,False,True,False,1933.0,2005-09-28,0.264,0.816,0.288,1.300,72.7255,-4.0,-4.0
4,011_S_0021,21,ADNI1,0.0,False,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,T

In [93]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["GroupN"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

/tmp/ipykernel_6292/2076110807.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_keys=False).apply(lambda x: x.loc[x["GroupN"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)


In [94]:
c.to_csv("clinical.csv")

In [95]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]

FileNotFoundError: [Errno 2] No such file or directory: 'overlap_test_set.csv'

In [ ]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "GroupN", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["GroupN"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
with open("X_train_c.pkl", 'wb') as X_tr:
    np.save(X_tr, X_train)
    
with open("Y_train_c.pkl", 'wb') as Y_tr:
    np.save(Y_tr, y_train)
    
with open("X_test_c.pkl", 'wb') as X_te:
    np.save(X_te, X_test)
    
with open("Y_test_c.pkl", 'wb') as Y_te:
    np.save(Y_te, y_test)
